## Семинар 1
# Оптимизация гиперпараметров

Гиперпараметр (англ. hyperparameter) — параметр, который не настраивается во время обучения модели. Пример гиперпараметра — шаг градиентного спуска, он задается перед обучением. Пример параметров — веса градиентного спуска, они изменяются и настраиваются во время обучения.

Попытаемся улучшить обобщающую способность модели, настроив ее гиперпараметры.
Задача оптимизации гиперпараметров заключается в подборе таких гиперпараметров, при которых заданная модель алгоритма будет наиболее эффективна.
Наиболее часто используемый метод – это решетчатый поиск (grid search), который перебирает все возможные комбинации интересующих гиперпараметров.

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Задания для выполнения:

### 1. Сгенерируйте данные для задачи бинарной классификации с 4 признаками и 10000 наблюдениями. Целевая переменная должна зависеть от 2 признаков. Выведите признаки в виде датафрейма, дав названия колонкам. Выведите число объектов в каждом классе.
Указание: Для этого воспользуйтесь [make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html), задав параметры: число строк n_samples=10000, число признаков n_features=4, число признаков от которых зависит целевая переменная n_informative=2, число избыточных признаков (т.е. признаков, являющихся линейными комбинациями признаков от которых зависит целевая переменная) n_redundant=2, фиксируем воспроизводимость случайных данных random_state=42.

In [40]:
X, y = datasets.make_classification(n_samples=10000,
                                    n_features=4,
                                    n_informative=2,
                                    n_redundant=2,
                                    random_state=42)

In [41]:
X[:5]

array([[ 1.52264971, -0.93455988, -0.46502165,  0.05887383],
       [ 1.04810323, -0.74680553,  0.43685274,  0.85962787],
       [ 1.02421601, -0.44941167, -1.62343415, -1.37865568],
       [-0.43466749,  0.2817535 ,  0.02329917, -0.13524348],
       [ 1.42116496, -0.73157419, -1.46292601, -1.05843534]])

In [42]:
df = pd.DataFrame(data = X, columns = ['X'+str(i) for i in range(1, X.shape[1] + 1)])
df.head()

,X1,X2,X3,X4
0,1.522650,-0.934560,-0.465022,0.058874
1,1.048103,-0.746806,0.436853,0.859628
2,1.024216,-0.449412,-1.623434,-1.378656
3,-0.434667,0.281754,0.023299,-0.135243
4,1.421165,-0.731574,-1.462926,-1.058435


### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 20% объектов и зафиксируйте способ перемешивания данных random_state=42. Выведите размеры обучающей и тестовой частей.

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape[0], X_test.shape[0]

(8000, 2000)

### 3. Постройте модель логистической регрессии с гиперпараметрами, установленными по умолчанию. Для этого создайте класс LogisticRegression c дефолтными гиперпараметрами и обучите модель на обучающих данных. Сделайте предсказание на тестовых данных. Вычислите метрику accuracy на тестовых данных.

In [44]:
default_model = LogisticRegression()
default_model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [45]:
default_model.fit(X_train, y_train)
accuracy_default = default_model.score(X_test, y_test)
accuracy_default

0.8935

### 4. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием hold-out разбиения, среди двух гиперпараметров, значения которых указанны ниже. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

Создадим диапазон вариантов для гиперпараметров в виде словаря:

In [46]:
param_grid = {'C': np.logspace(-3,2,9), 
              'solver': ['lbfgs','liblinear', 'saga']
}

In [47]:
def Grid(X, y):
    X_trainval, X_testval, y_trainval, y_testval = train_test_split(X, y, test_size=0.2, random_state=42)
    best_score, score = 0, 0
    for C in param_grid['C']:
        for solver in param_grid['solver']:
            model = LogisticRegression(C = C, solver = solver)
            model.fit(X_trainval, y_trainval)
            score = model.score(X_testval, y_testval)
            if score > best_score:
                best_score = score
                best_parameters = {'C': C, 'solver': solver}
    return  best_parameters

In [48]:
best_parameters = Grid(X_train, y_train)
best_parameters

{'C': 0.004216965034285823, 'solver': 'lbfgs'}

In [49]:
best_model = LogisticRegression(**best_parameters)
best_model.fit(X_train, y_train)
best_model.score(X_test, y_test)

0.894

### 5. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

In [50]:
def GridCV(X, y):
    best_score, score = 0, 0
    for C in param_grid['C']:
        for solver in param_grid['solver']:
            model = LogisticRegression(C = C, solver = solver)
            score = np.mean(cross_val_score(model, X, y, cv=5, scoring='accuracy'))
            if score > best_score:
                best_score = score
                best_parameters = {'C': C, 'solver': solver}
    return best_parameters

In [51]:
best_parameters = GridCV(X_train, y_train)
best_parameters

{'C': 0.01778279410038923, 'solver': 'saga'}

### 6. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании, используя класс GridSearchCV из библиотеки sklearn. Убедитесь, что результаты будут теми же, что и в предыдущем задании.
Указание: Для выведения наилучших гиперпараметров модели, используйте атрибут best_params_. Наилучшая модель выводится при помощи атрибута best_estimator_.

In [52]:
gs = GridSearchCV(
    estimator = LogisticRegression(),
    param_grid = param_grid,
    scoring = 'accuracy',
    cv = 5)

In [56]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                         'solver': ['lbfgs', 'liblinear', 'saga']},
             scoring='accuracy')

In [58]:
gs.best_params_

{'C': 0.01778279410038923, 'solver': 'saga'}

In [59]:
best_model_skl = LogisticRegression(**gs.best_params_)
best_model_skl.fit(X_train, y_train)
best_model_skl.score(X_test, y_test)

0.894

или

In [60]:
best_model_skl = gs.best_estimator_
best_model_skl.fit(X_train, y_train)
best_model_skl.score(X_test, y_test)

0.894

### 7. Выведите результаты решетчатого поиска с использованием класса GridSearchCV, осуществлённого в предыдущем пункте, в виде словаря и в виде датафрейма. Оставьте в датафрейме только два столбца со значениями гиперпараметров и со значениями вычисляемой метрики. Выведите самое большое значение среди средних значений метрики accuracy перекрестной проверки.
Указание: Для выведения результатов решетчатого поиска в виде словаря используйте атрибут cv_results_. Для выведения самого большого значения среди средних значений метрики accuracy используйте атрибут best_score_.

In [61]:
gs.cv_results_

{'mean_fit_time': array([0.00711594, 0.00437775, 0.00541348, 0.00252991, 0.00263176,
        0.00831318, 0.00248842, 0.00459304, 0.0055202 , 0.00247273,
        0.00343237, 0.00621967, 0.00236535, 0.00372772, 0.0055892 ,
        0.0028142 , 0.00364914, 0.00547218, 0.00367756, 0.00369415,
        0.00633378, 0.00251083, 0.00389514, 0.00625286, 0.00286555,
        0.00369539, 0.00651846]),
 'std_fit_time': array([0.00546336, 0.00173512, 0.00156459, 0.00092191, 0.00063749,
        0.00141734, 0.00177418, 0.00096135, 0.00111566, 0.00118099,
        0.00147835, 0.00076719, 0.00100803, 0.00157479, 0.00112856,
        0.00092746, 0.00134268, 0.00279724, 0.00237525, 0.00136342,
        0.00231298, 0.00131817, 0.00161896, 0.00139393, 0.00092469,
        0.00145538, 0.00175578]),
 'mean_score_time': array([0.00041342, 0.00048084, 0.00028644, 0.00039482, 0.0003859 ,
        0.00048738, 0.000281  , 0.00094481, 0.00037608, 0.0003222 ,
        0.00035615, 0.00039401, 0.00038681, 0.000315  , 0.000319

In [62]:
pd.DataFrame(gs.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007116,0.005463,0.000413,0.000076,0.001000,lbfgs,"{'C': 0.001, 'solver': 'lbfgs'}",0.88125,0.878125,0.886875,0.894375,0.895000,0.887125,0.006785,26
1,0.004378,0.001735,0.000481,0.000186,0.001000,liblinear,"{'C': 0.001, 'solver': 'liblinear'}",0.88375,0.878750,0.887500,0.891875,0.895000,0.887375,0.005761,25
2,0.005413,0.001565,0.000286,0.000047,0.001000,saga,"{'C': 0.001, 'solver': 'saga'}",0.88125,0.878125,0.886875,0.894375,0.895000,0.887125,0.006785,26
3,0.002530,0.000922,0.000395,0.000165,0.004217,lbfgs,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.88625,0.881875,0.891875,0.900000,0.896875,0.891375,0.006654,8
4,0.002632,0.000637,0.000386,0.000173,0.004217,liblinear,"{'C': 0.004216965034285823, 'solver': 'libline...",0.88750,0.883125,0.891875,0.900625,0.896250,0.891875,0.006187,4


In [63]:
pd.DataFrame({'params': gs.cv_results_['params'], 'mean_test_score': gs.cv_results_['mean_test_score']})

,params,mean_test_score
0,"{'C': 0.001, 'solver': 'lbfgs'}",0.887125
1,"{'C': 0.001, 'solver': 'liblinear'}",0.887375
2,"{'C': 0.001, 'solver': 'saga'}",0.887125
3,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.891375
4,"{'C': 0.004216965034285823, 'solver': 'libline...",0.891875
5,"{'C': 0.004216965034285823, 'solver': 'saga'}",0.891375
6,"{'C': 0.01778279410038923, 'solver': 'lbfgs'}",0.892375
7,"{'C': 0.01778279410038923, 'solver': 'liblinear'}",0.892250
8,"{'C': 0.01778279410038923, 'solver': 'saga'}",0.892500
9,"{'C': 0.07498942093324558, 'solver': 'lbfgs'}",0.891625


In [64]:
gs.best_score_

0.8924999999999998

### 8. Используйте методы score и predict класса GridSearchCV, чтобы вычислить метрики качества accuracy и F1-меры на тестовых данных для модели LogisticRegression с лучшими гиперпараметрами.

In [65]:
accuracy = gs.score(X_test, y_test)
accuracy

0.894

In [66]:
y_test_pred = gs.predict(X_test)
f1 = metrics.f1_score(y_test, y_test_pred)
f1

0.8922764227642277

### 9. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс GridSearchCV из библиотеки sklearn, среди списка словарей гиперпараметров, указанных ниже. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [67]:
param_grid = param_grid = [{'solver': ['liblinear'],
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3,2,9),
    'max_iter' : [1000]                        
   }, {
    'solver': ['saga'],
    'penalty': ['elasticnet'],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'max_iter' : [1000]
}]

In [68]:
gs_list = GridSearchCV(
    estimator = LogisticRegression(),
    param_grid = param_grid,
    scoring = 'accuracy',
    cv = 5)

In [69]:
gs_list.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                          'max_iter': [1000], 'penalty': ['l1', 'l2'],
                          'solver': ['liblinear']},
                         {'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
                          'max_iter': [1000], 'penalty': ['elasticnet'],
                          'solver': ['saga']}],
             scoring='accuracy')

In [72]:
gs_list.best_params_

{'C': 0.01778279410038923,
 'max_iter': 1000,
 'penalty': 'l1',
 'solver': 'liblinear'}

In [73]:
gs_list.score(X_test, y_test)

0.894

F1:

In [74]:
y_test_pred = gs_list.predict(X_test)
metrics.f1_score(y_test, y_test_pred)

0.8928210313447927

### 10. Осуществите рандомизированный решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс RandomizedSearchCV из библиотеки sklearn, для списка словарей гиперпараметров из предыдущего задания. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [75]:
rgs = RandomizedSearchCV(
    estimator = LogisticRegression(),
    param_distributions = param_grid,
    scoring = 'accuracy',
    cv = 5,
    random_state = 1)

In [76]:
rgs.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(),
                   param_distributions=[{'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                                         'max_iter': [1000],
                                         'penalty': ['l1', 'l2'],
                                         'solver': ['liblinear']},
                                        {'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
                                         'max_iter': [1000],
                                         'penalty': ['elasticnet'],
                                         'solver': ['saga']}],
                   random_state=1, scoring='accuracy')

In [77]:
pd.DataFrame({'params': rgs.cv_results_['params'], 'mean_test_score': rgs.cv_results_['mean_test_score']})

,params,mean_test_score
0,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890750
1,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.890875
2,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.891875
3,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.890875
4,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890750
5,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.890875
6,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.890750
7,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.893250
8,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.888750
9,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890875


# Задания для самостоятельного выполнения:

### 1. Загрузите встроенный датасет `load_iris`. Обозначьте признаки за `X`, а целевую переменную за `y`. Выведите первые 5 строк в виде датафрейма. 

### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 15% объектов и зафиксируйте способ перемешивания данных random_state=0. Выведите размеры обучающей и тестовой частей.

### 3. Для датасета load_iris осуществите классификацию методом логистической регрессии, создав класс LogisticRegression(max_iter = 1000). Выведите значения гиперпараметров этой модели, установленные по умолчанию. Обучите эту модель, сделайте предсказание. Оцените качество модели, выведя значения метрик accuracy и f1-score.

### 4. Оптимизируйте гиперпараметры, используя следующие наборы гиперпараметров: param_grid = [{'solver': ['lbfgs', 'newton-cg'],'penalty': ['l2']}, {'C': np.logspace(0,4,10), 'penalty': ['l2', 'l1'], 'solver': ['liblinear']}, {'C': np.logspace(0,4,10), 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},] по метрике accuracy. Реализуйте поиск лучших гиперпараметров с перекрёстной проверкой, используя класс GridSearchCV. Выведете результаты поиска в виде датафрейма, состоящего из двух колонок: в первой колонке все перебираемые сочитания гиперпараметров, а во второй соответствующие им средние значения метрики accuracy. Выведите значения лучших гиперпараметров.

### 5. Выведите значение метрики accuracy на тестовых данных. Cделайте предсказание для лучшей модели и выведите метрику f1-score. Сравните полученные метрики с метриками, полученными без оптимизации гиперпараметров.

### 6. Осуществите рандомизированный решетчатый поиск лучших гиперпараметров по метрике accuracy для той же самой сетки гиперпараметров. 

### 7. Осуществите решетчатый поиск лучших гиперпараметров по метрике f1 для той же самой сетки гиперпараметров. 